# Solving the Boltzman equation for DM Freeze-In

## Importing libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import mpmath 

## Usefull constants

In [3]:
mnu=0.0
pi=np.pi
G = 6.71*10**-45 # MeV
Mpl = (8 * pi * G) ** (-0.5)
yinf=4.35*10**-7 # Divided by the mass of the particle
y0=0.0
geff=3

## Import $g_{*s}$

In [4]:
data = np.loadtxt('g_*s(T).txt', skiprows=1)
t_imported = data[:, 0]
g_star_s_imported = data[:, 1]
def g_star_s_interp_func(t):
    return sp.interpolate.interp1d(t_imported, g_star_s_imported, fill_value="extrapolate")(t)


## Math functions

In [5]:
def bk(n,x):
    return sp.special.kn(n,x)

def gmeijer1(x):
    x_mpf = mpmath.mpf(x)
    try:
        return float(mpmath.meijerg([[], [1]], [[-1/2, -1/2, 1/2], []], x_mpf**2))
    except ValueError:
        return 0.0

def gmeijer2(x):
    x_mpf = mpmath.mpf(x)
    try:
        return float(mpmath.meijerg([[], [2]], [[-1/2, 1/2, 1/2], []], x_mpf**2))
    except ValueError:
        return 0.0

## Number density of massive fermions

In [6]:
def int_n_e(E, m, T):
    arg = E / T
    arg = np.clip(arg, None, 700)  # Avoid overflow in exp
    return E * np.sqrt(E**2 - m**2) / (np.exp(arg) + 1)

def n_e(m,T,glib):
    int=[]
    if m>0:
        int.append(sp.integrate.quad(int_n_e,m,m*10,args=(m,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        for i in range(10):
            int.append(sp.integrate.quad(int_n_e,m*10**(i+1),m*10**(i+2),args=(m,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        int1=np.sum(int)
        return (glib/(2*np.pi**2))*int1
    elif m==0:
        int.append(sp.integrate.quad(int_n_e,0,1,args=(0,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        for i in range(10):
            int.append(sp.integrate.quad(int_n_e,10**i,10**(i+1),args=(0,T),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
        int1=np.sum(int)
        return (glib/(2*np.pi**2))*int1
    else:
        print('Error: Negative mass')
        return 0
    

## Thermodinamical properties of "massless" fermions

In [7]:
def n_nu(T,glib):                                              # If Dirac fermions glib=4
    return 3*glib*sp.special.zeta(3)*T**3/(4*np.pi**2)         # If Majorana fermions glib=2 

def rho_nu(T,glib):                                                 # If Dirac fermions glib=4
    return (7*glib*np.pi**2*T**4)/(240)                        # If Majorana fermions glib=2 

def p_nu(T,glib):
    return (1/3)*rho_nu(T,glib)

def s_nu(T,glib):
    return (4*rho_nu(T,glib))/(3*T)

def Y_nu(T,glib):
    return n_nu(T,glib)/s_nu(T,glib)    

## Neutrino functions

$$ \sigma=\frac{g^2}{32\pi s^2}\sqrt{\frac{s-4m_\chi^2}{s}}\left[s+\frac{(s-4m_\chi^2)}{3}+4m_\chi^2    \right]$$

$$<\sigma v>=\frac{1}{8 m_\chi^4 T K_2^2(m_\chi/T)}\int_{4m_\chi^2}^\infty ds\; \sigma(s-4m_\chi^2)\sqrt{s}K_1(\sqrt{s}/T) $$

$$<\sigma v>=\frac{g^2}{192 \pi m^3 K_2^2(x)}\left\{4 m_\chi K_1^2(x)-\sqrt{\pi}m_\chi x\left[ 

G_1(x^2)+G_2(x^2)
\right]
\right\} $$

$$ \frac{dY_\nu}{dx}=\frac{2<\sigma v>_\nu n_\nu^2}{xHs}$$

In [8]:
def sigma_nu(s,g,mchi):
    a1=g**2/(32*pi*s**2)
    a2=np.sqrt((s-4*mchi**2)/(s))
    a3=(s+(s-4*mchi**2)/(3)+4*mchi**2)
    return a1*a2*a3

def intsigma_nu(s,g,mchi,T):
    a1=sigma_nu(s,g,mchi)
    a2=(s-4*mchi**2)
    a3=np.sqrt(s)
    a4=bk(1,np.sqrt(s)/T)
    return a1*a2*a3*a4

def sigmav_nu_num(g,mchi,x):
    int=[]
    n=50
    for i in range(n):
        int.append(sp.integrate.quad(intsigma_nu,2**(2*i+2)*mchi**2,2**(2*i+4)*mchi**2,args=(g,mchi,mchi/x),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
    int.append(sp.integrate.quad(intsigma_nu,2**(2*n+3)*mchi**2,np.inf,args=(g,mchi,mchi/x),epsabs=1e-10,epsrel=1e-10,limit=1000)[0])
    int2=np.sum(int)
    if int2==0.0:
        #print("Warning: sigmav_e is zero, returning 0")
        return 0.0
    a1=x/(8*mchi**5*bk(2,x)**2)
    return a1*(int2)

def dYdx_nu_num(y,x,g,mchi):
    a1=sigmav_nu_num(g,mchi,x)
    a2=n_nu(mchi/x,2)
    a3=((2*pi**2)/45)*(mchi/x)**3*g_star_s_interp_func(mchi/x)
    a4=(pi*(mchi/x)**2)*np.sqrt(g_star_s_interp_func(mchi/x)/90)/Mpl
    return 2*a1*a2**2/(x*a3*a4)

In [ ]:
TNU_OVER_TGAMMA_infty = (4.0/11.0)**(1.0/3.0)  # ~0.713766

def Tnu_over_Tgamma_approx(T_gamma_MeV, Tdec=0.511, width=0.6):
    x = np.array(T_gamma_MeV, dtype=float)
    s = np.tanh((x - Tdec) / width)  
    r = 0.5*(1.0 + s) * 1.0 + 0.5*(1.0 - s) * TNU_OVER_TGAMMA_infty
    return r

def Tnu_from_Tgamma(T_gamma_MeV, **kwargs):
    return Tnu_over_Tgamma_approx(T_gamma_MeV, **kwargs) * np.array(T_gamma_MeV, dtype=float)

In [ ]:
def sigmav_nu(g, mchi, x):
    bk2 = bk(2, x)
    denom = 192 * pi * mchi**3 * bk2**2
    num = (4 * mchi * bk(1, x)**2) - (np.sqrt(pi) * mchi * x * (gmeijer1(x) + gmeijer2(x)))
    return (g**2 / denom) * num

def dYdx_nu(y,x,g,mchi):
    T= mchi/x
    T_nu = Tnu_from_Tgamma(T)               # We change the temperature to T_nu for the yield computations but not the evolution of the universe
    x_nu = mchi/T_nu  
    a1=sigmav_nu(g,mchi,x_nu)
    a2=n_nu(mchi/x_nu,6)               # glib=6 for 3 neutrino species
    a3=((2*pi**2)/45)*(mchi/x)**3*g_star_s_interp_func(mchi/x)
    a4=(pi*(mchi/x)**2)*np.sqrt(g_star_s_interp_func(mchi/x)/90)/Mpl
    return 2*a1*a2**2/(x*a3*a4)

def dYdx_nu_mod(y,x,g,mchi):
    a1=sigmav_nu(g,mchi,x)
    a2=n_nu(mchi/x,6)               # glib=6 for 3 neutrino species
    a3=((2*pi**2)/45)*(mchi/x)**3*g_star_s_interp_func(mchi/x)
    a4=(pi*(mchi/x)**2)*np.sqrt(g_star_s_interp_func(mchi/x)/90)/Mpl
    return 2*a1*a2**2/(x*a3*a4)

In [11]:
mchi=[10]
x=np.logspace(-8,2,2000)
g_nu=np.logspace(-14,-11,100)  # Adjusted to match the range of mchi
temp1=[]
temp2=[]
contador=1


for i in range(len(mchi)):
    temp1=[]
    for j in range(len(g_nu)):
        print(f"Calculating for mchi={mchi[i]} MeV, g_C={g_nu[j]}           ", 100*contador/(len(g_nu)*len(mchi)),"%")
        contador=contador+1
        y_nu=sp.integrate.odeint(dYdx_nu,y0,x,args=(g_nu[j],mchi[i]),rtol=1e-11, atol=1e-11, mxstep=10000)
        temp1.append(y_nu)
    temp2.append(temp1)


Calculating for mchi=10 MeV, g_C=1e-14            1.0 %
Calculating for mchi=10 MeV, g_C=1.0722672220103253e-14            2.0 %
Calculating for mchi=10 MeV, g_C=1.1497569953977357e-14            3.0 %
Calculating for mchi=10 MeV, g_C=1.2328467394420684e-14            4.0 %
Calculating for mchi=10 MeV, g_C=1.3219411484660287e-14            5.0 %
Calculating for mchi=10 MeV, g_C=1.4174741629268077e-14            6.0 %
Calculating for mchi=10 MeV, g_C=1.5199110829529332e-14            7.0 %
Calculating for mchi=10 MeV, g_C=1.629750834620647e-14            8.0 %


/tmp/ipykernel_11576/1371211001.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  return (g**2 / denom) * num
/tmp/ipykernel_11576/1371211001.py:5: RuntimeWarning: invalid value encountered in scalar multiply
  return (g**2 / denom) * num


Calculating for mchi=10 MeV, g_C=1.747528400007683e-14            9.0 %
Calculating for mchi=10 MeV, g_C=1.8738174228603867e-14            10.0 %
Calculating for mchi=10 MeV, g_C=2.009233002565046e-14            11.0 %
Calculating for mchi=10 MeV, g_C=2.1544346900318866e-14            12.0 %
Calculating for mchi=10 MeV, g_C=2.310129700083158e-14            13.0 %
Calculating for mchi=10 MeV, g_C=2.477076355991714e-14            14.0 %
Calculating for mchi=10 MeV, g_C=2.656087782946684e-14            15.0 %
Calculating for mchi=10 MeV, g_C=2.848035868435805e-14            16.0 %
Calculating for mchi=10 MeV, g_C=3.053855508833412e-14            17.0 %
Calculating for mchi=10 MeV, g_C=3.2745491628777315e-14            18.0 %
Calculating for mchi=10 MeV, g_C=3.5111917342151274e-14            19.0 %
Calculating for mchi=10 MeV, g_C=3.7649358067924715e-14            20.0 %
Calculating for mchi=10 MeV, g_C=4.03701725859655e-14            21.0 %
Calculating for mchi=10 MeV, g_C=4.3287612810830

In [12]:
for j in range(len(mchi)):
    yvals = [temp2[j][i][-1] for i in range(len(g_nu))]
    xvals = [g_nu[i] for i in range(len(g_nu))]

    data = np.column_stack((xvals, yvals))

    # Keep only rows without NaN
    clean_data = data[~np.isnan(data).any(axis=1)]

    if clean_data.size == 0:
        print(f"All values are NaN for mchi={mchi[j]}, skipping.")
        continue

    np.savetxt(
        f"gNuvsgC_10.txt",
        clean_data,
        header="g_nu Final_Yield",
        fmt="%.6e"
    )